### Matrix Chain Multiplication

Determine the optimal parenthesization of a product of n matrices.

Find the most efficient way to multiply a given sequence of matrices. The problem is not actually to perform the multiplications but merely to decide the sequence of the matrix multiplications involved.

The matrix multiplication is associative as no matter how the product is parenthesized, the result will remain the same. For example, for four matrices A, B, C, and D, we would have:

```
((AB)C)D = ((A(BC))D) = (AB)(CD) = A((BC)D) = A(B(CD))
```

However, the order in which the product is parenthesized affects the number of simple arithmetic operations needed to compute the product or the efficiency. For example:

```
A is a 10x30 matrix
B is a 30x5 matrix
C is a 5x60 matrix

(AB)C needs (10 x 30 x 5) + (10 x 5 x 60) = 1500 + 3000 = 4500 operations

A(BC) needs (30 x 5 x 60) + (10 x 30 x 60) = 9000 + 180000 = 27000 operations

First method is more efficient
```

Break the problem into a set of related subproblems.

With recursion, we can try all possible combinations. We split the sequence into two subsequences and find the minimum cost of multiplying each subsequence. Add these costs together, and add in the price of multiplying the two resulting matrices. Repeat for every possible position the sequence can be split.

Given: array dims with dimensions such that each matrix M[i] has dimensions dims[i-1] x dims[i] for i = 1....n

Example:
```
[10, 30, 5, 60]
split into [10, 30] and [30, 5, 60]
[30, 5, 60] = 30 x 5 x 60 = 9000
Then [10, 30, 60] = 18000
9000 + 18000 = 27000

or split into [10, 30, 5] and [5, 60]
[10, 30, 5] = 10 x 30 x 5 = 1500
Then [10, 5, 60] = 3000
1500 + 3000 = 4500
```


In [1]:
def matrixMultiplication(dims, i=None, j=None):
    if i==None:
        i = 0
        j = len(dims) - 1
    # if there is just one matrix
    if j <= i + 1:
        return 0
    minimumOps = None
    for k in range(i+1, j):
        cost = matrixMultiplication(dims, i, k)
        cost += matrixMultiplication(dims, k, j)
        cost += dims[i] * dims[k] * dims[j]
        if minimumOps == None:
            minimumOps = cost
        else:
            minimumOps = min(minimumOps, cost)
    return minimumOps

In [100]:
%%time
matrixMultiplication([10,30,5,60,8,44,20,10,28,15,3,15,32,56,22,10,66])

CPU times: user 6.4 s, sys: 82.2 ms, total: 6.48 s
Wall time: 6.99 s


25218

In [4]:
def matrixChain(dims):
    minimumOps = None
    if len(dims) < 3:
        return 0
    if len(dims) == 3:
        return dims[0] * dims[1] * dims[2]
    for i in range(2, len(dims)):
        left = dims[:i]
        right = dims[i-1:]
        leftCount = matrixChain(left)
        rightCount = matrixChain(right)
        ops = leftCount + rightCount + (left[0] * left[-1] * right[-1])
        minimumOps = ops if minimumOps == None else min(minimumOps, ops)
    return minimumOps
            
            

In [102]:
%%time
matrixChain([10,30,5,60,8,44,20,10,28,15,3,15,32,56,22,10,66])


CPU times: user 4.71 s, sys: 34.5 ms, total: 4.74 s
Wall time: 4.87 s


25218

### Dynamic programming

Without recursion, we'll make a lookup table and calculate the number of operations for the smallest possible subsequences first. From there we'll fill out to the larger subsequences, taking the minimum operations required by adding the smaller subsequences first and then calculating the operations needed to multiply the resulting matrix with the one we're adding.

For example:

```

[10, 30, 5, 60]

We'll use i to track the starting position and j to track the stopping position.
We'll start by tracking the multiplication of two matrices, meaning i and j can only be 2 apart.

i = 0
j = 2

Our lookup tracks i on the y axis, j on the x axis. 10 * 30 * 5 = 1500, put that at lookup[0][2]


            10            30             5               60
            
   10       0             0            1500               0
   
   30       0             0              0                0
   
   5        0             0              0                0 
   
   60       0             0              0                0
   
Next is i=1 and j=3, that's 30 * 5 * 60 = 9000, put that at lookup[1][3]


            10            30             5               60
            
   10       0             0            1500               0
   
   30       0             0              0               9000
   
   5        0             0              0                0 
   
   60       0             0              0                0
   
Now we can start looking at longer subsequences. We'll start with i=0, j=3.

We'll assign k to see how we should split up this sequence. Start with k at 1, meaning that we'll consider the 10x30 matrix separately and apply it only after we've figured out everything from i=1 to j=3.

k = 1
lookup[i][k] = 0
lookup[k][j] = 9000
lookup[i][j] = 0 + 9000 + dimensions[i] * dimensions[k] * dimsensions[j] = 18000

k = 2
lookup[i][k] = 1500
lookup[k][j] = 0
lookup[i][j] = 1500 + 0 + dimsensions[i] * dimensions[k] * dimensions[j] = 4500

Since 4500 < 18000, we'll take the 4500



            10            30             5               60
            
   10       0             0            1500              4500
   
   30       0             0              0               9000
   
   5        0             0              0                0 
   
   60       0             0              0                0
   


```


In [108]:
10 * 5 * 60


3000

In [105]:
matrixChainLookup([10,30,5,60])

[[0, 0, 1500, 4500], [0, 0, 0, 9000], [0, 0, 0, 0], [0, 0, 0, 0]]


4500

In [104]:
def matrixChainLookup(dims):
    lookup = [[0] * len(dims) for _ in range(len(dims))]
    for diff in range(2, len(dims)):
        # we're going to fill in the lookup table with all the possible first pairings of matrices, then work up
        for i in range(len(dims) - 2):
            j = i + diff
            if j == i + 2:
                lookup[i][j] = dims[i] * dims[i+1] * dims[j]
              
            elif j < len(dims):
                k = i + 1
                lookup[i][j] = lookup[i][k] + lookup[k][j] + (dims[i] * dims[k] * dims[j])
               
                k += 1
                while k < j:
                    lookup[i][j] = min(lookup[i][j], lookup[i][k] + lookup[k][j] + (dims[i] * dims[k] * dims[j]))
                    k += 1
    print(lookup)
    return lookup[0][-1]

In [101]:
%%time
matrixChainLookup([10,30,5,60,8,44,20,10,28,15,3,15,32,56,22,10,66])

CPU times: user 948 µs, sys: 9 µs, total: 957 µs
Wall time: 954 µs


25218

In [68]:
matrixChainLookup([1,2,3,4,5,6])


i 0 k 2 j 3 18
i 1 k 3 j 4 64
i 2 k 4 j 5 150
i 0 k 2 j 4 81
i 0 k 3 j 4 38
i 1 k 3 j 5 192
i 1 k 4 j 5 124
i 0 k 2 j 5 174
i 0 k 3 j 5 162
i 0 k 4 j 5 68


[[0, 0, 6, 18, 38, 68],
 [0, 0, 0, 24, 64, 124],
 [0, 0, 0, 0, 60, 150],
 [0, 0, 0, 0, 0, 120],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]